## Additive attention
Paper: [Neural machine translation by jointly learning to align and translate](https://arxiv.org/pdf/1409.0473) - Bahdanau et. al 2015

Dataset: [Multi30K English to Deutsche dataset](https://huggingface.co/datasets/bentrevett/multi30k)

Model: Use LSTM as encoder and decoder

#### Model variation
- BiLSTM encoder + LSTM decoder

This is the additive attention implementation which reuses lot of components built from the seq2seq training.

In [2]:
try:
    import google.colab
    is_running_on_colab = True
except ImportError:
    is_running_on_colab = False

In [3]:
# colab specific
%pip install early-stopping-pytorch

/Users/niranjan/Documents/programming/AI/deeplearning/projects/deeplearning-practice/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import random
import numpy as np
import torch

SEED = 1557
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [13]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from utils.multi30k_data_processing_utils import word_tokenize, build_vocab, preprocess, decode, collate_fn
from utils.multi30k_data_processing_utils import BOS, PAD, EOS

train_dataset = load_dataset("bentrevett/multi30k", split="train")
tokenized_train_dataset = train_dataset.map(lambda x: {"en": word_tokenize(x["en"]), "de": word_tokenize(x["de"])}, batched=False)

en_vocab_to_index, en_index_to_vocab = build_vocab(
    [item["en"] for item in tokenized_train_dataset]
)
de_vocab_to_index, de_index_to_vocab = build_vocab(
    [item["de"] for item in tokenized_train_dataset]
)

preprocessed_train_dataset = train_dataset.map(
    lambda x: preprocess(x, "de", "en", de_vocab_to_index, en_vocab_to_index),
    batched=True,
    remove_columns=["en", "de"]
)
preprocessed_train_dataset.set_format(type="torch", columns=["source", "target"])

print(preprocessed_train_dataset[0])
print(f"German: {decode(de_index_to_vocab, preprocessed_train_dataset[0]['source'].tolist())}")
print(f"English: {decode(en_index_to_vocab, preprocessed_train_dataset[0]['target'].tolist())}")

loader = DataLoader(
    preprocessed_train_dataset,
    batch_size=3,
    shuffle=True,
    collate_fn=collate_fn,
)

batch = next(iter(loader))
print(batch["source"].shape)
print(batch["source_lengths"].shape)
print(batch["target"].shape)
print(batch["target_lengths"].shape)

Map:   0%|          | 0/29000 [00:00<?, ? examples/s]

{'source': tensor([   1,   18,   27,  215,   31,   85,   20,   89,    7,   15,  115,    3,
        3149,    4,    2]), 'target': tensor([   1,   16,   24,   15,   25,  776,   17,   57,   80,  204, 1305,    5,
           2])}
German: zwei junge weiße männer sind im freien in der nähe <unk> büsche .
English: two young , white males are outside near many bushes .
torch.Size([3, 29])
torch.Size([3])
torch.Size([3, 23])
torch.Size([3])


In [6]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout, device, padding_idx=0):
        super().__init__()
        self.device = device
        self.directions = 2
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, dropout=dropout if num_layers > 1 else 0.0, batch_first=True, bidirectional=True)
        self.hidden_projection = nn.Linear(2 * hidden_dim, hidden_dim)
        self.cell_projection = nn.Linear(2 * hidden_dim, hidden_dim)

    def forward(self, source_encodings, source_lengths): # source_encodings: (batch_size, max_length), source_lengths: (batch_size)
        B, T = source_encodings.size()
        h_0 = torch.zeros(self.num_layers * self.directions, B, self.hidden_dim, device=self.device)
        c_0 = torch.zeros(self.num_layers * self.directions, B, self.hidden_dim, device=self.device)

        embedded = self.embedding(source_encodings) # (B, T, embedding_dim)
        embedded = self.dropout(embedded)

        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, source_lengths.to('cpu'), batch_first=True, enforce_sorted=False)
        packed_outputs, (last_hidden, last_cell) = self.lstm(packed_embedded, (h_0, c_0)) # hidden: (num_layers * directions, B, hidden_dim)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs, batch_first=True) # outputs: (B, T, hidden_dim * directions)

        h_f = last_hidden[0::2] # (num_layers, B, hidden_dim)
        h_b = last_hidden[1::2] # (num_layers, B, hidden_dim)
        h_cat = torch.cat((h_f, h_b), dim=2) # (num_layers, B, hidden_dim * 2) concatenate along the hidden dimension

        c_f = last_cell[0::2] # (num_layers, B, hidden_dim)
        c_b = last_cell[1::2] # (num_layers, B, hidden_dim)
        c_cat = torch.cat((c_f, c_b), dim=2) # (num_layers, B, hidden_dim * 2) concatenate along the hidden dimension

        last_hidden = torch.tanh(self.hidden_projection(h_cat)) # (num_layers, B, hidden_dim)
        last_cell = torch.tanh(self.cell_projection(c_cat)) # (num_layers, B, hidden_dim)

        return outputs, last_hidden, last_cell


In [7]:
from torch.nn import functional as F

class BahdanauAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.query_projection = nn.Linear(hidden_dim, hidden_dim)
        self.key_projection = nn.Linear(2 * hidden_dim, hidden_dim) # bidirectional LSTM
        self.score_projection = nn.Linear(hidden_dim, 1, bias=False)


    def forward(self, decoder_hidden_states, encoder_outputs): # decoder_hidden_states: (num_layers, B, hidden_dim), encoder_outputs: (B, T, 2 * hidden_dim) bidirectional LSTM
        # use the last hidden state of the multi-layer LSTM decoder as the query
        query = decoder_hidden_states[-1] # (B, hidden_dim)
        query = query.unsqueeze(1) # (B, 1, hidden_dim)

        query_projected = self.query_projection(query) # (B, 1, hidden_dim)
        keys_projected = self.key_projection(encoder_outputs) # (B, T, hidden_dim)

        # Note: query_projected is broadcasted to (B, T, hidden_dim) automatically without expand or repeat by PyTorch
        energy = self.score_projection(torch.tanh(query_projected + keys_projected)).squeeze(-1) # (B, T)

        # take a mask to prevent the attention to the padding tokens

        attention_weights = F.softmax(energy, dim=1) # (B, T)

        return attention_weights

In [8]:
class AttentionDecoder(nn.Module):
    """
    Decoder for the Seq2Seq model. This works on a batch of single step targets (B, 1).
    It doesn't take the entire target sequence, because the decision to teacher force and what kind of search strategy to use
    is done at the sequence level, not the step level.
    """
    def __init__(self, attention, vocab_size, embedding_dim, hidden_dim, num_layers, dropout, padding_idx=0):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        self.dropout = nn.Dropout(dropout)
        self.attention = attention
        self.lstm = nn.LSTM(
             2 * hidden_dim + embedding_dim,
            hidden_dim,
            num_layers,
            dropout=dropout if num_layers > 1 else 0.0,
            batch_first=True,
            bidirectional=False,
        )
        self.output_projection = nn.Linear(hidden_dim, vocab_size)

    def forward(self, inputs, encoder_last_hidden, encoder_last_cell, encoder_outputs):
        # Note: (B, 1) is needed because the LSTM expects a 2D input (batch_size, sequence_length)
        inputs = inputs.unsqueeze(1) # (B, 1)

        embedded = self.embedding(inputs)  # (B, 1, embedding_dim)
        embedded = self.dropout(embedded)

        attention_weights = self.attention(encoder_last_hidden, encoder_outputs) # (B, T)
        attention_weights = attention_weights.unsqueeze(1) # (B, 1, T)
        context_vector = attention_weights @ encoder_outputs # (B, 1, hidden_dim)

        lstm_input = torch.cat((embedded, context_vector), dim=2) # (B, 1, embedding_dim + 2 *hidden_dim)

        outputs, (hidden, cell) = self.lstm(lstm_input, (encoder_last_hidden, encoder_last_cell)) # (B, 1, hidden_dim)
        outputs = self.dropout(outputs)
        logits = self.output_projection(outputs) # (B, 1, vocab_size)

        return logits, hidden, cell, attention_weights.squeeze(1)

In [9]:
test_encoder = Encoder(len(de_vocab_to_index), 256, 256, 2, 0.0, 'cpu')
test_attention = BahdanauAttention(256)
test_decoder = AttentionDecoder(test_attention, len(en_vocab_to_index), 256, 256, 2, 0.0, de_vocab_to_index[BOS])

encoder_outputs, hidden, cell = test_encoder.forward(batch["source"], batch["source_lengths"])

logits, hidden, cell, attention_weights = test_decoder.forward(batch["target"][:, 0], hidden, cell, encoder_outputs)

print(f"target: {batch['target'][:, 0]}")
print(f"Logits: {logits.shape}")
print(f"Hidden: {hidden.shape}")
print(f"Cell: {cell.shape}")
print(f"Attention weights: {attention_weights.shape}")

print(f"Attention weights: {attention_weights}")


target: tensor([1, 1, 1])
Logits: torch.Size([3, 1, 4560])
Hidden: torch.Size([2, 3, 256])
Cell: torch.Size([2, 3, 256])
Attention weights: torch.Size([3, 15])
Attention weights: tensor([[0.0664, 0.0667, 0.0671, 0.0665, 0.0663, 0.0667, 0.0666, 0.0664, 0.0663,
         0.0668, 0.0662, 0.0670, 0.0675, 0.0670, 0.0668],
        [0.0672, 0.0668, 0.0664, 0.0675, 0.0677, 0.0673, 0.0672, 0.0662, 0.0662,
         0.0662, 0.0662, 0.0662, 0.0662, 0.0662, 0.0662],
        [0.0668, 0.0669, 0.0671, 0.0667, 0.0669, 0.0672, 0.0665, 0.0669, 0.0663,
         0.0661, 0.0663, 0.0664, 0.0668, 0.0670, 0.0663]],
       grad_fn=<SqueezeBackward1>)


In [10]:
class AttentionSeq2Seq(nn.Module):
    def __init__(self, encoder, decoder, bos_idx, eos_idx, max_target_length, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.bos_idx = bos_idx
        self.eos_idx = eos_idx
        self.max_target_length = max_target_length
        self.device = device

    def forward(self, source, source_lengths, target=None, topk=0):
        B = source.shape[0]

        encoder_outputs, hidden, cell = self.encoder(source, source_lengths)

        if target is not None:
            steps = target.shape[1] - 1
            inputs = target[:, 0]
        else:
            steps = self.max_target_length
            inputs = torch.full((B,), self.bos_idx, dtype=torch.long, device=self.device)

        logits_all = torch.zeros(B, steps, self.decoder.vocab_size, device=self.device)
        preds_all = torch.zeros(B, steps, device=self.device)

        for t in range(steps):
            logits, hidden, cell, attention_weights = self.decoder(inputs, hidden, cell, encoder_outputs)
            step_logits = logits.squeeze(1) # (B, 1, vocab_size) -> (B, vocab_size)
            if topk > 0:
                step_preds = self._topk_decode(step_logits, topk)
            else:
                step_preds = step_logits.argmax(dim=1)

            logits_all[:, t, :] = step_logits
            preds_all[:, t] = step_preds

            # Teacher forcing
            if target is not None:
                inputs = target[:, t + 1]
            else:
                inputs = step_preds
                if step_preds.eq(self.eos_idx).all():
                    break

        return logits_all, preds_all, attention_weights

    def _topk_decode(self, logits, topk):
        step_logits = logits.squeeze(1) # (B, vocab_size)

        topk_logits, topk_vocab_ids = torch.topk(step_logits, topk, dim=-1) # (B, k)
        topk_probs = torch.softmax(topk_logits, dim=-1) # (B, k)

        sample_pos = torch.multinomial(topk_probs, num_samples=1) # (B, 1)
        preds = topk_vocab_ids.gather(1, sample_pos) # (B, 1)
        return preds.squeeze(1)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [ ]:
tgt_vocab_size = len(en_vocab_to_index) # 4560
src_vocab_size = len(de_vocab_to_index) # 5422
emb_dim = 512
hidden_dim = 512
enc_num_layers = 2
dec_num_layers = 2
enc_dropout = 0.3
dec_dropout = 0.3
padding_idx = de_vocab_to_index[PAD]
bos_idx = de_vocab_to_index[BOS]
eos_idx = de_vocab_to_index[EOS]
max_target_length = 30

batch_size = 128
learning_rate = 0.0005
epochs = 30
max_grad_norm = 1.0
topk = 5
early_stopping_patience = 5

In [16]:
encoder = Encoder(src_vocab_size, emb_dim, hidden_dim, enc_num_layers, enc_dropout, device, padding_idx)
additive_attention = BahdanauAttention(hidden_dim)
decoder = AttentionDecoder(additive_attention, tgt_vocab_size, emb_dim, hidden_dim, dec_num_layers, dec_dropout, padding_idx)

model = AttentionSeq2Seq(encoder, decoder, bos_idx, eos_idx, max_target_length, device)
model.to(device)

AttentionSeq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5422, 512, padding_idx=0)
    (dropout): Dropout(p=0.3, inplace=False)
    (lstm): LSTM(512, 512, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
    (hidden_projection): Linear(in_features=1024, out_features=512, bias=True)
    (cell_projection): Linear(in_features=1024, out_features=512, bias=True)
  )
  (decoder): AttentionDecoder(
    (embedding): Embedding(4560, 512, padding_idx=0)
    (dropout): Dropout(p=0.3, inplace=False)
    (attention): BahdanauAttention(
      (query_projection): Linear(in_features=512, out_features=512, bias=True)
      (key_projection): Linear(in_features=1024, out_features=512, bias=True)
      (score_projection): Linear(in_features=512, out_features=1, bias=False)
    )
    (lstm): LSTM(1536, 512, num_layers=2, batch_first=True, dropout=0.3)
    (output_projection): Linear(in_features=512, out_features=4560, bias=True)
  )
)

In [18]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The Seq2Seq model has {count_parameters(model):,} trainable parameters')
print(f"Encoder parameters: {count_parameters(encoder):,}")
print(f"Decoder parameters: {count_parameters(decoder):,}")
print(f"Attention parameters: {count_parameters(additive_attention):,}")

The Seq2Seq model has 26,089,424 trainable parameters
Encoder parameters: 14,327,808
Decoder parameters: 11,761,616
Attention parameters: 787,968


In [19]:
criterion = nn.CrossEntropyLoss(ignore_index=de_vocab_to_index[PAD]).to(device)

In [21]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [23]:
from utils.train_utils import train, validate
from utils.multi30k_data_processing_utils import collate_fn

In [24]:
train_dataset = load_dataset("bentrevett/multi30k", split="train")
val_dataset = load_dataset("bentrevett/multi30k", split="validation")

final_train_dataset = train_dataset.map(
    lambda x: preprocess(x, "de", "en", de_vocab_to_index, en_vocab_to_index),
    batched=True,
    remove_columns=["en", "de"]
)
final_train_dataset.set_format(type="torch", columns=["source", "target"])

final_val_dataset = val_dataset.map(
    lambda x: preprocess(x, "de", "en", de_vocab_to_index, en_vocab_to_index),
    batched=True,
    remove_columns=["en", "de"]
)
final_val_dataset.set_format(type="torch", columns=["source", "target"])

pin_memory = True if device == "cuda" else False

train_loader = DataLoader(final_train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=2, pin_memory=pin_memory, persistent_workers=True)
val_loader = DataLoader(final_val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=2, pin_memory=pin_memory, persistent_workers=True)

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

In [ ]:
import time
from early_stopping_pytorch import EarlyStopping
from utils.train_utils import epoch_time

# inital test
epochs = 3

def train_loop(model, train_loader, val_loader, criterion, optimizer, epochs, model_path, patience=3):
    early_stopping = EarlyStopping(patience=patience, verbose=True)
    train_losses = []
    val_losses = []

    best_valid_loss = float('inf')

    for e in range(epochs):
        start_time = time.time()
        train_loss = train(model, train_loader, criterion, optimizer)
        val_loss = validate(model, val_loader, criterion)

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        if val_loss < best_valid_loss:
            best_valid_loss = val_loss
            torch.save(model.state_dict(), model_path)

        train_losses.append(train_loss)
        val_losses.append(val_loss)

        print(f"Epoch {e+1}: Train loss: {train_loss:.4f}, Val loss: {val_loss:.4f}, Epoch time: {epoch_mins}m {epoch_secs}s")

        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break
    return train_losses, val_losses

In [26]:
if is_running_on_colab:
    from google.colab import drive
    drive.mount('/content/drive')

    model_location = '/content/drive/My Drive/ML study/Attentions/additive attention'
    artifacts_location = model_location
else:
    model_location = 'weights'
    artifacts_location = 'artifacts'


In [ ]:
model_path = f'{model_location}/additive-attention-model.pt'

train_losses, val_losses = train_loop(model, train_loader, val_loader, criterion, optimizer, epochs, model_path, patience=early_stopping_patience)

In [ ]:
from utils.train_utils import save_artifacts, show_graph

artifacts_path = f'{artifacts_location}/additive-attention-model-artifacts.json'
save_artifacts(model, train_losses, val_losses, batch_size, learning_rate, epochs, artifacts_path, model_path)
show_graph(train_losses, val_losses, save_path=f'{artifacts_location}/additive-attention-model-loss-graph.png', save=True)

In [ ]:
from utils.train_utils import load_model
load_model(model, f'{model_location}/additive-attention-model.pt', device)

In [ ]:
test_dataset = load_dataset("bentrevett/multi30k", split="test")
test_dataset = test_dataset.map(
    lambda x: preprocess(x, "de", "en", de_vocab_to_index, en_vocab_to_index),
    batched=True,
    remove_columns=["en", "de"]
)
test_dataset.set_format(type="torch", columns=["source", "target"])

test_loader = DataLoader(test_dataset, batch_size=5, shuffle=True, collate_fn=collate_fn)

In [ ]:
from utils.train_utils import batch_to_device

@torch.no_grad()
def translate(model, test_loader, criterion, device):
    model.eval()

    one_batch = next(iter(test_loader))
    one_batch = batch_to_device(one_batch, device)

    targets = one_batch["target"]
    logits_all, preds_all, attention_weights = model(one_batch["source"], one_batch["source_lengths"], topk=topk)

    for i in range(preds_all.shape[0]):
        print(f"Source: {decode(de_index_to_vocab, one_batch['source'][i].tolist())}")
        print(f"Pred: {decode(en_index_to_vocab, preds_all[i].tolist())}")
        print(f"Target: {decode(en_index_to_vocab, targets[i].tolist())}")
        print(f"Attention weights: {attention_weights[i]}")
        print("-"*100)

In [ ]:
translate(model, test_loader, criterion, device)